In [24]:
using Pkg
include("./P2H_CapacityExpansion.jl")
cd("/cluster/home/danare/git")
Pkg.activate(".")
using .P2H_CapacityExpansion
using JuMP, MathOptAI
using DataFrames
using ScikitLearn
using JuMP
using Ipopt
using MathOptAI
using Parameters

  Activating project at `~/git`


# 1. Read in the data

In [9]:
file = "/cluster/home/danare/git/P2H_CapacityExpansion/results/aggregated_results/500_scenarios.txt"

"/cluster/home/danare/git/P2H_CapacityExpansion/results/aggregated_results/500_scenarios.txt"

In [10]:
df = P2H_CapacityExpansion.read_txt_file(file);
size(df)

(1391, 13)

# 2. Split the Data into Training and Test

In [11]:
X_train, y_train, X_test, y_test = P2H_CapacityExpansion.partitionTrainTest(df, :Cost, 0.8)

### scale the data ###
X_train_scaled, μX, σX  = P2H_CapacityExpansion.scaling(X_train)
X_test_scaled = (X_test .- μX) ./ σX
y_train_scaled, μy, σy  = P2H_CapacityExpansion.scaling(y_train)
    
# remove np.nan #
for i in eachindex(X_test_scaled)
    if isnan(X_test_scaled[i])
        X_test_scaled[i] = 0.0
    end
end

# 3. Train the Model

In [12]:
### train ML model and compute R2 ### 
sg = P2H_CapacityExpansion.neural_network_model_flux(X_train, y_train, X_test, y_test; hidden_layer=500, epochs=1000)

Epoch = 1 : Training loss = 4477.265735886937, R2 score : -0.1075189418118685
Epoch = 2 : Training loss = 4666.618991447867, R2 score : -0.10314121529647391
Epoch = 3 : Training loss = 4384.643487633775, R2 score : -0.09166871894016348
Epoch = 4 : Training loss = 4263.7224153860225, R2 score : -0.07875303492545616
Epoch = 5 : Training loss = 4019.186645233374, R2 score : -0.060220234469183564
Epoch = 6 : Training loss = 3788.941315944246, R2 score : -0.0338087262505542
Epoch = 7 : Training loss = 3686.355745698158, R2 score : -0.010893152511065596
Epoch = 8 : Training loss = 3580.14036805574, R2 score : 0.010668380663022559
Epoch = 9 : Training loss = 3538.3563376533257, R2 score : 0.02726994874705535
Epoch = 10 : Training loss = 3437.268941946542, R2 score : 0.049943405449826894
Epoch = 11 : Training loss = 3369.190586625243, R2 score : 0.07150092358139959
Epoch = 12 : Training loss = 3346.5887908521345, R2 score : 0.09033549768096372
Epoch = 13 : Training loss = 3242.3296565973073, R

Main.P2H_CapacityExpansion.Surrogate(Chain(Dense(12 => 500, relu), Dense(500 => 1)), [2703.004150390625, 1217.3284912109375, 1451.1793212890625, 2535.618408203125, 1284.1793212890625, 1828.0203857421875, 1358.8826904296875, 1972.9832763671875, 480.77734375, 1978.3953857421875  …  1281.7955322265625, 1421.1868896484375, 480.59344482421875, 2426.528076171875, 1922.0726318359375, 2087.947509765625, 43686.04296875, 10802.0322265625, 1332.9617919921875, 32554.541015625])

# 4. Hybrid Modeling Approach

In [47]:
# load input
config = P2H_CapacityExpansion.read_yaml_file();
data = P2H_CapacityExpansion.load_cep_data(config=config);
ts_data = P2H_CapacityExpansion.load_timeseries_data_full(config=config);

┌ Info: Each time-series is averaged in 10-hourly steps
└ @ Main.P2H_CapacityExpansion /cluster/home/danare/git/P2H_CapacityExpansion/utils/load_data.jl:76


In [43]:
cep = P2H_CapacityExpansion.run_opt(ts_data=ts_data, data=data, config=config, surrogate=true, solver=Ipopt.Optimizer)

@unpack 𝓖, 𝓨, 𝓣, 𝓡, 𝓢, 𝓛, 𝓒 = P2H_CapacityExpansion.get_sets(cep=cep)
data = data.data;

##################### cost optimization #####################

P2H_CapacityExpansion.setup_opt_costs_fix!(cep, config, data,vcat(cep.sets["non_dispatch"], cep.sets["dispatch"], 𝓢, String[s for s in cep.sets["discharging"]], cep.sets["conversion"]))

#JuMP.fix.(cep.model[:COST]["var", :, :], 0; force=true);
@variable(cep.model, COST_VAR[y ∈ 𝓨] ≥ 0);

┌ Info: Reading the data ...
└ @ Main.P2H_CapacityExpansion /cluster/home/danare/git/P2H_CapacityExpansion/src/opt.jl:19
┌ Info: Investment mode is on.
└ @ Main.P2H_CapacityExpansion /cluster/home/danare/git/P2H_CapacityExpansion/src/opt.jl:88


In [44]:
techs = [f for f ∈ cep.sets["invest_tech"] if f != "ENS"]

for y ∈ 𝓨, r ∈ 𝓡
    x_vec = [cep.model[:TotalCapacityAnnual][r, g, y] for g ∈ techs]
    prediction, formulation = MathOptAI.add_predictor(cep.model, sg.model, x_vec)
    @constraint(cep.model, COST_VAR[y] .>= prediction)
end

In [45]:
opex_discounted = sum(
    1 / ((1 + config["r"])^(y - 𝓨[1] - 10)) * (
        sum(cep.model[:COST]["fix", y, g] for g ∈ 𝓖) +
        COST_VAR[y]
    ) for y ∈ 𝓨
)

@objective(cep.model, Min, sum(
    1 / ((1 + config["r"])^(y - 𝓨[1] - 10)) *
    sum(cep.model[:COST]["cap", y, g] for g ∈ cep.sets["invest_all"])
    for y ∈ 𝓨 
) + opex_discounted)

1.4802442849183444 COST[cap,2030,P_Wind_Offshore_Transitional] + 1.4802442849183444 COST[cap,2030,S_Battery_Li-Ion] + 1.4802442849183444 COST[cap,2030,P_Wind_Onshore_Avg] + 1.4802442849183444 COST[cap,2030,X_Electrolysis] + 1.4802442849183444 COST[cap,2030,trans_H2] + 1.4802442849183444 COST[cap,2030,P_PV_Utility_Avg] + 1.4802442849183444 COST[cap,2030,P_Gas_CCGT] + 1.4802442849183444 COST[cap,2030,D_Gas_H2_out] + 1.4802442849183444 COST[cap,2030,trans_elec] + 1.4802442849183444 COST[cap,2030,D_Battery_Li-Ion_out] + 1.4802442849183444 COST[cap,2030,ENS] + 1.4802442849183444 COST[cap,2030,P_Biomass] + 1.4802442849183444 COST[cap,2030,S_Gas_H2] + 1.4802442849183444 COST[cap,2030,P_H2_OCGT] + 1.4802442849183444 COST[cap,2030,X_ATR_CCS] + COST[cap,2040,P_Wind_Offshore_Transitional] + COST[cap,2040,S_Battery_Li-Ion] + COST[cap,2040,P_Wind_Onshore_Avg] + COST[cap,2040,X_Electrolysis] + COST[cap,2040,trans_H2] + COST[cap,2040,P_PV_Utility_Avg] + COST[cap,2040,P_Gas_CCGT] + COST[cap,2040,D_Gas

In [ ]:
result = P2H_CapacityExpansion.optimize_and_output(cep=cep, config=config, data=data, ts_data=ts_data, name="scenario_v3", short_sol=false)

In [50]:
prediction

1-element Vector{VariableRef}:
 moai_Affine[1]

In [36]:
println("Total variables: ", JuMP.num_variables(cep.model))
println("Total constraints: ", JuMP.num_constraints(cep.model))

Total variables: 6369


UndefKeywordError: UndefKeywordError: keyword argument `count_variable_in_set_constraints` not assigned

In [53]:
typeof(z)

VariableRef (alias for GenericVariableRef{Float64})